In [ ]:
from __future__ import division
from pyoperators import pcg
from pysimulators import profile

# QUBIC packages
import qubic
from qubicpack.utilities import Qubic_DataDir
from qubic.data import PATH
from qubic.io import read_map
from qubic import QubicSkySim as qss
import os
path = os.getcwd()

path_mm = os.path.dirname(path)
import sys
sys.path.append(path_mm)


import frequency_acquisition as Acq
#import component_acquisition

# Display packages
import healpy as hp
import matplotlib.pyplot as plt

# FG-Buster packages
import component_model as c
import mixing_matrix as mm

# General packages
import numpy as np
import pysm3
import warnings
from qubic import QubicSkySim as qss
import pysm3.units as u
from importlib import reload
from pysm3 import utils

from importlib import reload
import gc
from pyoperators import *
from pysimulators import *


from pysimulators.interfaces.healpy import HealpixConvolutionGaussianOperator
warnings.filterwarnings("ignore")

def integration(nu_min, nu_max, Nintegr, sky_config, d, fwhm_subbands = None):
    print(f'Integration from {nu_min:.2f} to {nu_max:.2f} GHz with {Nintegr} steps')
    obj = Acq.QubicIntegrated(d, Nsub=Nintegr, Nrec=Nintegr)
    if Nintegr == 1:
        allnus = np.array([np.mean([nu_min, nu_max])])
    else:
        allnus = np.linspace(nu_min, nu_max, Nintegr)
    m = obj.get_PySM_maps(sky_config, nus=allnus)
    
    if fwhm_subbands is not None:
        for i in range(Nintegr):
            C = HealpixConvolutionGaussianOperator(fwhm=fwhm_subbands[i])
            m[i] = C(m[i])
    
    return np.array([np.mean(m, axis=0)])


# # %config InlineBackend.figure_format='retina'
from IPython.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
rc('figure',figsize=(20,8))


In [ ]:
reload(Acq)
nside = 256
relative_bandwidth = 0.25
band = 220
band_planck = 217
noise = False
level_noise_planck = 1
npointings = 200


allnftod = np.arange(2,102, 10)
alltod = np.zeros((len(allnftod), npointings))

for i in range(len(allnftod)):
    Nf_TOD = allnftod[i]
    Nf_recon = 5
    fact_sub = 4
    convolution = True

    reload(Acq)


    dictfilename = 'dicts/pipeline_demo.dict'
    d = qubic.qubicdict.qubicDict()
    d.read_from_file(dictfilename)

    d['nf_recon'] = Nf_recon
    d['nf_sub'] = Nf_recon*fact_sub
    d['nside'] = nside
    npix=12*d['nside']**2
    d['RA_center'] = 0
    d['DEC_center'] = -57
    center = qubic.equ2gal(d['RA_center'], d['DEC_center'])
    d['effective_duration'] = 3
    d['npointings'] = npointings
    d['filter_nu'] = band * 1e9
    d['photon_noise'] = noise
    d['noiseless'] = not noise
    d['config'] = 'FI'
    d['filter_relative_bandwidth'] = 0.25
    d['MultiBand'] = True
    d['planck'] = True
    d['dtheta'] = 15
    d['synthbeam_dtype'] = float

    d['nhwp_angles'] = 1

    print('*************** Noise **************')
    print('Noisless:      {}'.format(d['noiseless']))
    print('Photon Noise:  {}'.format(d['photon_noise']))
    print('************************************')

    sky_config = {'cmb':42}#, 'dust':'d0'}

    reload(Acq)

    qubic_acquisition = Acq.QubicIntegrated(d, Nsub=Nf_TOD, Nrec=Nf_TOD)

    m_sub = qubic_acquisition.get_PySM_maps(sky_config, qubic_acquisition.allnus)
    ### no polarization
    m_sub[:,:,1:2] = 0
    ### a map with ony ones
    m_sub[:,:,0] = 1

    array = qubic_acquisition._get_array_of_operators(convolution=False, myfwhm=None)
    h_tod = BlockRowOperator(array, new_axisin=0)
    tod = h_tod(m_sub)

    mytod = tod[445,:]
    alltod[i,:] = mytod

    del h_tod
    gc.collect()


In [ ]:
rc('figure',figsize=(20,8))
subplot(1,3,1)
for i in range(len(allnftod)):
    plot(alltod[i,:], label=allnftod[i])
legend()
    
subplot(1,3,2)
allrms = np.zeros(len(allnftod))
for i in range(len(allnftod)):
    plot(alltod[i,:] - alltod[-1,:], label=allnftod[i])
    allrms[i] = np.std(alltod[i,:] - alltod[-1,:])
legend()

subplot(1,3,3)
plot(allnftod, allrms/allrms[0], 'o-')
yscale('log')


In [ ]:
allnftod